# Deep Learning

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
print(tf.__version__)
keras.__version__

2.12.0


'2.12.0'

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full,y_train_full), (X_test,y_test) = fashion_mnist.load_data()

In [ ]:
X_train_full.shape, X_train_full.dtype

In [ ]:
X_valid, X_train = X_train_full[:5000]/255.0, X_train_full[5000:]/255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test/255.0

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
class_names[y_train[0]]

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(300,activation="relu"),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(10,activation="softmax")
])
model.summary()

# ***Erklärung der Schichten***
1. Flatten: nimmt die Eingabe und macht daraus ein 1D Array
2. Dense: 300 Neuronen, mit ReLU Aktivierungsfunktion
    Die erste Sicht hat bspweise 
    
                                28 x 28 = 784 

                                784 x 300 + 300 Bias-Terme = 235.500
                                
    was sich zu 235.500 Parameter ausfummiert. Jedoch Risiko overfitting.
3. Dense: 100 Neuronen, mit ReLU Aktivierungsfunktion
4. Dense: 10 Neuronen, mit Softmax Aktivierungsfunktion


### Liste mit den sichten eines Modells erhalten, um eine davon über ihren Index anzusprechen, oder Sie gehen über den Namen vor:

In [ ]:
model.layers

In [ ]:
hidden1 = model.layers[1]
hidden1.name

In [ ]:
model.get_layer(hidden1.name) is hidden1

In [ ]:
weights, biases = hidden1.get_weights() # get_weights() and set_weights() are used to get and set the weights of a layer
print(biases)
biases.shape

#### Es ist gut das alle biases auf 0 gesetzt sind. Da wir das so möchten um die Symeterie zu erhalten

## ***Das Model compilen und trainieren***

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])


- sparse_categorical_crossentropy wird verwendet, weil wir spärliche Beschriftungen haben (d.h. für jede Instanz gibt es nur einen Zielklassenindex, in diesem Fall von 0 bis 9), und die Klassen sind exklusiv.
- Hätten wir stattdessen eine Zielwahrscheinlichkeit pro Klasse für jede Instanz (wie z.B. One-Hot-Vektoren, z.B. [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], um Klasse 3 zu repräsentieren), dann müssten wir stattdessen den categorical_crossentropy loss verwenden.
- Wenn wir eine binäre Klassifizierung durchführen (mit einem oder mehreren binären Labels), dann würden wir in der Ausgabeschicht anstelle der Softmax-Aktivierungsfunktion die sigmoide (d.h. logistische) Aktivierungsfunktion verwenden und den binary_crossentropy loss einsetzen.
- Wenn wir regressieren würden, dann würden wir keine Aktivierungsfunktion in der Ausgabeschicht verwenden, und wir würden den Verlust mean_squared_error verwenden.
- Der Optimierer ist der Algorithmus, der zur Aktualisierung der Gewichte des Modells verwendet wird. Der gebräuchlichste Optimierer ist der Stochastische Gradientenabstieg (SGD), aber es gibt noch viele andere, und einige von ihnen sind viel schneller als SGD. Einige Optimierer sind auch in der Lage, die Lernrate während des Trainings automatisch zu verringern, so dass Sie sie nicht manuell einstellen müssen. Dies ist der Fall beim Adam-Optimierer, der sehr beliebt ist.
- Das Argument metrics wird verwendet, um die zu berechnenden Metriken anzugeben. In diesem Fall geht es nur um die Genauigkeit.



In [ ]:
history = model.fit(X_train,y_train, epochs=30, validation_data=(X_valid,y_valid))

In [ ]:
history.params
history.history.keys()

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1) # vertikale Bereich auf (0,1) setzten
plt.show()

In [ ]:
model.evaluate(X_test,y_test)

## Vorhersagen Treffen

In [ ]:
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)


In [ ]:
# Assuming model is your trained Keras model
y_pred_probabilities = model.predict(X_new)
y_pred_classes = np.argmax(y_pred_probabilities, axis=-1)
np.array(class_names)[y_pred_classes]

In [ ]:
y_new = y_test[:3]
y_new

# ***Das Gradientenverfahren***

- Allgemeiner Algoritmus zur Optimierung, der optimale Lösungen für eine Vielzahl von Fragestellungen ermitteln kann.
Der Grundgedanke beim Gradientenverfahren ist, die Parameter iterativ so zu verändern das eine Kostenfunktion minimiert wird.

# Gegen instable Gradienten kämpfen 
- Hyperparameter 
    - Droputout
    - recurrent_dropout
abschwächung der instablien Gradienten und das RNN effizienter Trainieren.


### ***Batch Normalization nur bei der Eingabe also Input Layer***

#### Das Problem des Kurzzeitgedächtnisses

Aufgrund der Transformation, die die Daten auf ihrem Weg durch ein RNN durchlaufen, gehen bei jedem Zeitschritt manche Informationen verloren. Nach einiger Zeit enthält der Status RNN keinerlei Spuren der ursprünglichen Eingaben mehr. 
>BSP: Stellen wir uns vor jemand muss ein sehr langen Satz übersetzten - wenn Sie ihn fertig gelesen haben weiß sie nicht mehr, womit er begann.
Um das Problem zu beheben wurde eine Langzeitgedächtnis vorgestellt mit sehr großem erfolg.

Long Short Term Memory (LSTM)
- Wie eine Blackbox vorstellen. 
- Training wird schneller Konvergieren und wird langfrisitge Abhängigkeiten erkennen in Daten.

# ***Grundlegende ideen bei einem Neural Network***


Ein Deep Neural Network (DNN) ist eine leistungsstarke Technik des maschinellen Lernens, die in verschiedenen Bereichen Anwendung findet. Die Bedeutung eines Deep Neural Networks liegt in seiner Fähigkeit, komplexe Muster in Daten zu erkennen und Vorhersagen oder Klassifikationen basierend auf diesen Mustern zu treffen.


 Hier sind einige der Schlüsselaspekte, die die Bedeutung von DNNs verdeutlichen:


- 1. Mustererkennung: DNNs sind in der Lage, Muster in großen und komplexen Datensätzen zu erkennen (die nicht Linear sind), die von Menschen schwer oder unmöglich zu identifizieren sind. Dies macht sie besonders wertvoll in Anwendungen wie Bilderkennung, Spracherkennung und Natural Language Processing (NLP).


- 2. Klassifikation und Vorhersage: DNNs ermöglichen die Klassifikation von Daten in verschiedene Kategorien oder die Vorhersage zukünftiger Ereignisse. Dies wird in Anwendungen wie Spam-Erkennung, Gesichtserkennung, medizinischer Diagnose und Finanzprognosen genutzt.


- 3. Skalierbarkeit: DNNs können große Mengen an Daten verarbeiten und sind in der Lage, Modelle mit Tausenden oder Millionen von Parametern zu trainieren. Dies ermöglicht die Bewältigung komplexer Aufgaben und die Verarbeitung großer Datensätze.


- 4. Automatisierung: DNNs können automatisch und kontinuierlich lernen, was bedeutet, dass sie sich an neue Daten und Veränderungen in der Umgebung anpassen können, ohne dass eine manuelle Anpassung erforderlich ist.


- 5. Vielseitigkeit: DNNs finden in verschiedenen Bereichen Anwendung, darunter Bildverarbeitung, Sprachverarbeitung, autonome Fahrzeuge, medizinische Diagnostik, Empfehlungssysteme und vieles mehr. Sie sind eine Kernkomponente von Künstlicher Intelligenz (KI) und Deep Learning.


- 6. Forschung und Innovation: DNNs haben zu bedeutenden Fortschritten in der KI-Forschung geführt und neue Möglichkeiten für innovative Anwendungen eröffnet. Sie sind ein Motor für die Entwicklung neuer Technologien und Lösungen.


Die Bedeutung eines Deep Neural Networks liegt in seiner Fähigkeit, komplexe Aufgaben zu bewältigen und menschenähnliche Leistungen bei der Verarbeitung von Informationen zu erzielen. Dies hat Auswirkungen auf viele Branchen und trägt zur Transformation von Geschäftsmodellen und zur Lösung komplexer Herausforderungen bei.

--

## ***Blackbox***: 

- Als Blockbox wir in Data Science die Hiddenlayers gemeint die der Programmier nicht sehen. Da nicht nachvollziebar ist wie das Model den Output generiert. Das Auch Deepl Learning Neurale Netzwerke Hidden Layers nutzen und die für den Programmierer auch eine Blackbox ist (siehe bild)

![Hidden Layer](/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/hiddenlayer.png)


### ***Arten von rekurrenten neuronalen Netzwerken (RNNs)***

Eine Eingabesequenz ist einer Ausgabe zugeordnet. Sie können sie jedoch flexibel an verschiedene Konfigurationen für bestimmte Zwecke anpassen. Im Folgenden sind einige gänige RNN-Typen aufgeführt.

#### **Eins-zu-viele**
- Dieser RNN-Typ kanalisiert einen Eingang an mehrere Ausgänge. Es ermöglicht sprachliche Anwendungen wie Bildunterschirften, indem ein Satz aus einem einzigen Schüsselwort generiert wird.

#### **Viel-zu-viele**
- Das Modell verwendet mehrere Eingaben, um mehrere Ausgaben vorherzusagen. Sie können beispielsweise einen Sprachübersetzter mit einem RNN erstellen, der einen Satz analysiert und die Wörter in einer anderen Sprache korrekt strukturiert.

#### **Viel-zu-eins**
- Mehrere Eingänge werden einem Ausgang zugeordet. Dies ist hilfreich bei Anwendungen wie der Stimmungsanalyse, bei der das Modell positive, negative und neutrale Stimmungen der Kunden anhand von Kundenreferenzen vorhersagt.


<img src="/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/rnn_models.jpeg" width="500" />



### **Rekurrentes neuronales Netzwerk im Vergleich zu neuronalem Feed-Forward-Netzwerk**

- Wie Rnns sind neuronale Feed-Forward-Netze künstliche neuronale Netze, die Informationen von einem Ende zum Ende der Architektur weiterleiten. 
Ein neuronales Feed-Forward-Netzwerk kann einfache Klassifizierungens-, Regressions- oder Erkennungsaufgaben ausführen, aber es kann sich nicht an die vorherige Eingabe erinnern, die es verarbeitet hat.

- Zumbeispiel vergisst es "Apfel", wenn sein Neuron das Wort *ist* verarbeitet.
Das RNN überwindet diese Speicherbeschränkung, indem es einen verborgenen Gedächtniszustand in das Neuron einbezieht.

# ***Backpropagation***


Was macht Backpropagation ?
- Bei jedem Trainingsdatenpunkt trifft der Backpropagation-Algorithmus zuerst eine Vohersage (im Vorwärtsdurchlauf), bestimmt den Fehler, durchläuft dann rückwärst jede Sicht, um den Fehlerbeitrag jeder Verbindung zuermitteln (im Rückwärtsdurchlauf), und verändert schließlich die Gewichte der Verbindungen, um den Fehler zu verringern (als Schritt im Gradientenverfahren)
    - Es ist wichtig, alle Verbindungsgewichte der verborgenen Sichten mit Zufallswerte zu initialisieren, denn sonst wird das Training fehlschlagen. Initialisieren Sie zum Beispiel alle Gewichte und Bias mit null, werden alle Neuronen in einer Schicht identisch sein, und die Backpropagation wird sie exakt gleich behandeln, womit sie identisch bleiben.      


# ***Weights and Bias***
- In einem einfachen neuronalen Netzwerk wird die Berechnung ähnlich wie bei einer linearen Regression durchgeführt, bei der eine Ausgabe 
**y** anhand einer linearen Kombination der Eingaben **x**, eines Intercepts (Gewichts) und eines Slopes (Bias) berechnet wird. Der wesentliche Unterschied besteht darin, dass ein neuronales Netzwerk aus mehreren Schichten von Neuronen besteht, die miteinander verbunden sind und nichtlinear miteinander interagieren. Diese **nichtlineare Interaktion** wird durch die Verwendung einer **Aktivierungsfunktion** in jedem Neuron ermöglicht. Dadurch ist das neuronale Netzwerk in der Lage, **komplexere Beziehungen** zwischen den Eingaben und der Ausgabe zu erfassen und zu modellieren.


<img src="/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/Terminologie_weights.png" width="400" />

<img src="/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/Terminologie_baises.png" width="400" />


--

**Activation Function apply a nonlinear transformation and decide whether a neuron should be activated or not**


- Ohne eine Aktivierungsfunktion ist es eine stacked Lineare Funktion zulösen. *Die Aktiverungsfunktion unbetten die funktion des Rnns*
- wenn wir jedoch ein nicht Lineares Problem haben, brauchen wir eine nicht Lineare aktiverungs funktion

Es gibt viele aktivierungsfunktionen für ein Neurales Network
- Step Function
- TanH
- Leaky ReLU
- Softmax
- ReLU (Rectified Linear Unit)
- Softplus
- Sigmoid 
- Die Funktion **ReLU** ist eine Aktivierungs-funktion, die in neuronalen Netzten verwendet wird. Sie steht für **(Rectified Linear Unit)**. 
Diese Funktion gibt einfach den Eingabewert zurück, **wenn er positiv ist, und gibt 0 zurück, wenn er negativ ist**. Es ist eine weit verbreitete Aktivierungsfunktion aufgrund ihrer Einfachheit und ihrer guten Leistung in vielen Anwendungen.

![Arten von Aktiverierungsfunktion](/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/aktivierungsfunktionen.jpeg)

Formel :


![Beschreibung des Bildes](/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/eq_29.png)



- y(t) = Steht für die Ausgabe des neuronalen Netzes zur Zeit (t). 
Es repräsentiert das, was das Netzwerk zu einem bestimmten Zeitpunkt vorhersagt oder generiert.


- **Wx** und **Wy** sind Gewichtsmatrizen, die die Verbindungen zwischen den Eigaben **Xt** und den vorherigen Ausgaben **yt−1** und den versteckten Schichten des neuronalen Netzes steuern. WXT und WTY bezeichne die Transponierten dieser Gewichtsmatrizen.

- **Xt** ist die Eingabe des neuronales Netzes zum Zeitpunkt **t**. Sie repräsentiert die Daten oder Informationen, die dem Netzwerk zu diesem Zeitpunkt präsentiert werden.

- **yt-1** ist die Ausgabe des neuronalen Netzes zum vorherigen Zeitpunkt t - 1. Sie wird verwendet, um die Informationen aus vergangenen Zustände zu berücksichtigen.

- **b** ist der Bias-Vektor für die versteckten Schichten des neuronalen Netzes, ER ermöglicht es dem Netzwerk, Verschiebungen in den Daten zu berücksichtigen.

Insgesamt wird die Ausgabe des neuronalen Netzes zum Zeitpunkt **t**
als gewichtete Summe der Eingaben **Xt** und der vorherigen Ausgabe **yt-1** berechnet, die dann durch **die ReLU-Aktivierungsfunktion** gehen. Dieser Prozess wird verwendet, um Vorhersagen zu machen oder Daten zu generieren, basierend auf den Informationen zu diesem Zeitpunkt und vergangenen Zuständen des Netzwerks.

### ***Deep Learning ist ein Oberbegriff für Maschinenlernverfahren, die auf neuronalen Netzen mit mehreren Schichten basieren.
### Rekurrente neuronale Netzwerke (RNNs) sind eine spezielle Art von neuronalen Netzen, die für die Verarbeitung sequenzieller Daten entwickelt wurden und eine Rückkopplungsschleife haben, um Kontextinformationen über die Zeit hinweg zu speichern und zu verarbeiten.
### RNNs sind ein spezifisches Beispiel für Deep Learning-Modelle, die in bestimmten Anwendungsfällen besonders gut funktionieren, wie z.B. bei der Verarbeitung von Zeitreihen oder Textdaten.***


die Fähigkeit, Informationen über die Zeit hinweg zu speichern und zu verwenden. Diese Eigenschaft macht RNNs besonders gut geeignet für die Verarbeitung von sequenziellen Daten wie Zeitreihen, Texten oder Audiodaten.

# Kapitel 15

## 1. Verarbeiten von Sequenzen mit Rnns und Cnns

- RNN = Rekurrente neuronale Netze
CNN = 

Folgendes wird im Kaptiel Thematisert
1. Grundlegende Konzepte auf den RNN aufbaut und werden erfahren wie man sie per Backpropagation durch die Zeit trainiert
2. Schwierigkeiten der Modelle Thematisieren
    - Instablie Gradienten die durch verschiedene Techniken abgemildert werden können - unter anderem Recurrent Dropout und Recurrent Layer Normalization
    - Ein sehr begrenztes Kurzeitgedächtnis das mithilfe von LSTM und GRU-Zellen erweiter werden kann

Rnn sind nicht die einzigen Neuronal Netz die sequenzielle Daten verarbeiten können

In [3]:
import numpy as np
import keras
import pandas as pd

In [4]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2,offsets1, offsets2 = np.random.rand(4,batch_size, 1)
    time = np.linspace(0,1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))    # Welle 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))   # Welle 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)     # + Rauschen
    return series[..., np.newaxis].astype(np.float32)

In [5]:
n_steps = 50

series = generate_time_series(10000, n_steps + 1)
series.shape

(10000, 51, 1)

In [6]:
# Erstellungs eines Traininigs einen Validierungs und einen Testdatensattz
n_steps = 50

series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000: -1]


In [7]:
print("Wir haben also 10000 Reihen mit 50 Spalten", series.shape)

Wir haben also 10000 Reihen mit 50 Spalten (10000, 51, 1)


In [8]:
print("Der X Test Datensatz besthet aus: ", X_train.shape, "Der Y Test Datensatz besthet aus: ", y_train.shape)
print("Der X Validierungs Datensatz besthet aus: ", X_valid.shape,"Der Validierungs Datensatz besthet aus: ", y_valid.shape)
print("Der X_test Datensatz besthet aus: ", X_test.shape,"Der y_test Datensatz besteht  aus: ", y_test.shape)


Der X Test Datensatz besthet aus:  (7000, 50, 1) Der Y Test Datensatz besthet aus:  (7000, 1)
Der X Validierungs Datensatz besthet aus:  (2000, 50, 1) Der Validierungs Datensatz besthet aus:  (2000, 1)
Der X_test Datensatz besthet aus:  (1000, 50, 1) Der y_test Datensatz besteht  aus:  (999, 51, 1)


In [9]:
print("**Daten typ**", type(X_train), type(y_train), type(X_valid), type(y_valid), type(X_test), type(y_test))
# alle sind numpy arrays und haben die gleiche Anzahl an Dimensionen
print("Shape der Daten",X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape)

**Daten typ** <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
Shape der Daten (7000, 50, 1) (7000, 1) (2000, 50, 1) (2000, 1) (1000, 50, 1) (999, 51, 1)


In [10]:
y_pred = X_valid[:,-1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.020702371

# ***Mean squared Error MSE***
Der mittlere quadratische Fehler (MSE) ist eine gängige Metrik zur Bewertung der Leistung von Regressionsmodellen. Sie misst die durchschnittliche quadratische Abweichung zwischen den tatsächlichen und den vorhergesagten Werten.

Hier ist eine schrittweise Erklärung des MSE:
1. Differenz zwischen tatsächlichem und vorhergesagtem Wert: Für jede Instanz in den Daten wird die Differenz zwischen dem tatsächlichen Wert 
​und dem vorhergesagten Wert berechnet.

2. Quadrat der Differenz: Die Differenzen werden quadriert. Dies stellt sicher, dass negative und positive Abweichungen gleich behandelt werden und dass größere Abweichungen stärker ins Gewicht fallen.

3. Durchschnitt der quadrierten Differenzen: Die quadrierten Differenzen werden dann gemittelt, um den MSE zu erhalten. Dies geschieht, indem man die Summe der quadrierten Differenzen durch die Anzahl der Instanzen teilt.

**Mathematisch ausgedrückt:**


<img src="/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/mse.png" width="400" />



wo: 
n die anzahl 
yi der tatsächliche Wert den i-ten Instanz ist
y^i der vorherergesagte Wert der i-ten Instanz ist.


**WICHTIG:**
- Ein niedriger MSE-Wert deutet drauf hin, dass das Modell gute Vorhersagen macht, da die tatsächlichen und vorhergesagten Werte eng beieinander liegen. 

- Ein hoher MSE-Wert deutet hingegen darauf hin, dass das Model schlechte Vorhersagen macht, da die Abweichung zwischen den tatäschlich und den vorhergesagten Werten groß ist.


1. Flatten-Schicht: ist also wenn ich mir das Modul vorstellen das Input des Models
Vorschläge anzeigen

Die Flatten-Schicht kann als "Eingangstrichter" des Modells betrachtet werden. Sie nimmt die Eingabedaten entgegen und formatiert sie so, dass sie von den folgenden Schichten des Modells verarbeitet werden können.

**Details:**

Die Flatten-Schicht nimmt Daten mit einer beliebigen Form entgegen und "ebnet" sie in einen eindimensionalen Vektor.
Dies ist notwendig, da die meisten anderen Schichten des Modells mit Daten in Form von Vektoren arbeiten.
Die Größe des Vektors hängt von der Anzahl der Merkmale in den Eingabedaten ab.
Beispiel:

Angenommen, Sie haben Eingabedaten mit der Form (50, 10). Dies bedeutet, dass Sie 50 Datensätze mit jeweils 10 Merkmalen haben.

Die Flatten-Schicht würde diese Daten in einen Vektor mit 500 Elementen umwandeln.
Jedes Element des Vektors würde einem Merkmal aus einem der 50 Datensätze entsprechen.
Vorsichtsmaßnahmen:

Die Flatten-Schicht kann nur für Daten verwendet werden, die bereits in Form eines Arrays vorliegen.
Wenn Ihre Daten in einem anderen Format vorliegen, müssen Sie sie vor der Verwendung der Flatten-Schicht in ein Array konvertieren.

**Erklärung des Code abschnitt:**

- **keras.Models.Sequential** leitet ein model ein
- **keras.layer.Flatten** ist das Input, wenn man sich das Model vorsellt.
- **keras.layer.Dense** ist das Output des Models. 

    - In unseren konrekten code ist noch keine Hidden layer eingebaut
Somit können wir sehen das wir nur ein Input sowie Output haben des RNN Models. Jedoch noch keine Hiddenlayer

In [11]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50,1], name = "Input"),
    keras.layers.Dense(1, name = "Output")
])


 In diesem Fall wird **mean_squared_error** verwendet, was den mittleren quadratischen Fehler berechnet. Diese Funktion eignet sich gut für Regressionsaufgaben, bei denen kontinuierliche Werte vorhergesagt werden sollen.

In [12]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [13]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
  1/219 [..............................] - ETA: 29s - loss: 0.2866

2024-03-06 16:33:23.883244: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


219/219 [==============================] - 0s 317us/step - loss: 0.0888
Epoch 2/20
219/219 [==============================] - 0s 322us/step - loss: 0.0310
Epoch 3/20
219/219 [==============================] - 0s 308us/step - loss: 0.0179
Epoch 4/20
219/219 [==============================] - 0s 274us/step - loss: 0.0127
Epoch 5/20
219/219 [==============================] - 0s 299us/step - loss: 0.0103
Epoch 6/20
219/219 [==============================] - 0s 414us/step - loss: 0.0088
Epoch 7/20
219/219 [==============================] - 0s 271us/step - loss: 0.0076
Epoch 8/20
219/219 [==============================] - 0s 284us/step - loss: 0.0067
Epoch 9/20
219/219 [==============================] - 0s 278us/step - loss: 0.0059
Epoch 10/20
219/219 [==============================] - 0s 273us/step - loss: 0.0053
Epoch 11/20
219/219 [==============================] - 0s 279us/step - loss: 0.0049
Epoch 12/20
219/219 [==============================] - 0s 277us/step - loss: 0.0046
Epoch 13/20


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (Flatten)             (None, 50)                0         
                                                                 
 Output (Dense)              (None, 1)                 51        
                                                                 
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________


Loss: ist ein Maß dafür wie gut oder schlecht Ihr Modell während des Trainings abschneidet. 

**model.compile(loss='mean_squared_error', optimizer='adam')** auch (MSE) genannt.
In meinem Fall habe ich als *loss funktion* den **'mean_squared_error'** eingesetzt.

- Höhe des Verlustes: Ein niedriger Verlustwert bedeutet, dass Ihr Modell gute Vorhersagen macht und die tatsächlichen Werte gut reproduziert. Ein hoher Verlustwert deutet darauf hin, dass Ihr Modell schlechte Vorhersagen macht und die tatsächlichen Werte nicht gut reproduziert.

- Veränderung des Verlustes über die Epochen: Wenn der Verlust im Laufe des Trainings abnimmt, deutet dies darauf hin, dass Ihr Modell besser wird und die Vorhersagen genauer werden. Wenn der Verlust jedoch stagniert oder steigt, kann dies darauf hinweisen, dass Ihr Modell nicht mehr lernt oder überangepasst ist.

## Ein einfaches RNN implementieren
### SimpleRnn

In [15]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape = [None, 1])
])

In [16]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [17]:
model.fit(X_train, y_train,  epochs=20)

Epoch 1/20
219/219 [==============================] - 1s 3ms/step - loss: 0.2785 - accuracy: 0.0000e+00
Epoch 2/20
219/219 [==============================] - 1s 2ms/step - loss: 0.1962 - accuracy: 0.0000e+00
Epoch 3/20
219/219 [==============================] - 1s 2ms/step - loss: 0.1588 - accuracy: 0.0000e+00
Epoch 4/20
219/219 [==============================] - 1s 2ms/step - loss: 0.1432 - accuracy: 0.0000e+00
Epoch 5/20
219/219 [==============================] - 1s 2ms/step - loss: 0.1403 - accuracy: 0.0000e+00
Epoch 6/20
219/219 [==============================] - 1s 3ms/step - loss: 0.1398 - accuracy: 0.0000e+00
Epoch 7/20
219/219 [==============================] - 1s 3ms/step - loss: 0.1399 - accuracy: 0.0000e+00
Epoch 8/20
219/219 [==============================] - 1s 3ms/step - loss: 0.1407 - accuracy: 0.0000e+00
Epoch 9/20
219/219 [==============================] - 1s 3ms/step - loss: 0.1398 - accuracy: 0.0000e+00
Epoch 10/20
219/219 [==============================] - 1s 2ms/st

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 1)                 3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


## Deep Rnns
hier ist es ein Deep Rnn da wir ein hidden layers nutzen. Jedoch erstmal eine Linear Funktion da wir noch keine Aktivierungsfunktion haben.

In [19]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape = [None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

In [20]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [21]:
model.fit(X_train, y_train,  epochs=20)

Epoch 1/20
219/219 [==============================] - 3s 9ms/step - loss: 0.0419
Epoch 2/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0073
Epoch 3/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0050
Epoch 4/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0041
Epoch 5/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0037
Epoch 6/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0036
Epoch 7/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0032
Epoch 8/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0033
Epoch 9/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0031
Epoch 10/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0031
Epoch 11/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0032
Epoch 12/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0031
Epoch 13/20
219/219 [====

In [22]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape = [None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

In [23]:
model.compile(loss = 'mean_squared_error',optimizer='adam')

In [24]:
model.fit(X_train, y_train,  epochs=20)

Epoch 1/20
219/219 [==============================] - 2s 6ms/step - loss: 0.0119
Epoch 2/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0039
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0032
Epoch 4/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0031
Epoch 5/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0030
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0030
Epoch 7/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0030
Epoch 8/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0029
Epoch 9/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0029
Epoch 10/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0029
Epoch 11/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0028
Epoch 12/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0029
Epoch 13/20
219/219 [====

In [25]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_4 (SimpleRNN)    (None, None, 20)          440       
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 20)                820       
                                                                 
 dense (Dense)               (None, 1)                 21        
                                                                 
Total params: 1,281
Trainable params: 1,281
Non-trainable params: 0
_________________________________________________________________


In [26]:
series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:] # X_new ist die Zeitreihe bis 50 und Y_new ist die Zeitreihe ab 50
X = X_new
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)

Y_pred = X[:, n_steps:]
# Berechnung des MSE


1/1 [==============================] - 0s 11ms/step


In [27]:
np.mean(keras.losses.mean_squared_error(Y_new, Y_pred))

0.005400221

### Seite 513

In [28]:
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]  # 10 Schritte in die Zukunft
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [29]:
series.shape, X_train.shape, Y_train.shape, X_valid.shape, Y_valid.shape, X_test.shape, Y_test.shape

((10000, 60, 1),
 (7000, 50, 1),
 (7000, 10),
 (2000, 50, 1),
 (2000, 10),
 (1000, 50, 1),
 (1000, 10))

In [30]:
model = keras.models.Sequential([ 
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape = [None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

In [31]:
Y_pred = model.predict(X_new)

1/1 [==============================] - 0s 137ms/step


In [32]:
model.compile(loss = 'mean_squared_error',optimizer='adam')

In [33]:
model.fit(X_train, y_train,  epochs=20)

Epoch 1/20
219/219 [==============================] - 2s 6ms/step - loss: 0.1666
Epoch 2/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1463
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1457
Epoch 4/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1456
Epoch 5/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1452
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1452
Epoch 7/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1451
Epoch 8/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1451
Epoch 9/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1450
Epoch 10/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1450
Epoch 11/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1449
Epoch 12/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1449
Epoch 13/20
219/219 [====

In [34]:
y_pred = X_valid[:,-1]
np.mean(keras.losses.mean_squared_error(Y_valid, Y_pred))

0.51333916

# Seite 514

In [35]:
Y = np.empty((10000, n_steps, 10)) # jedes Ziel ist eine Sequenz von 10-D-Vektoren
for step_ahead in range(1, 10 + 1):
    Y[:,:, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0]  
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]


In [36]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape = [None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])
model.compile(loss = 'mean_squared_error',optimizer='adam')
model.fit(X_train, Y_train,  epochs=20)

Epoch 1/20
219/219 [==============================] - 2s 6ms/step - loss: 0.0655
Epoch 2/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0405
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0365
Epoch 4/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0347
Epoch 5/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0336
Epoch 6/20
219/219 [==============================] - 2s 7ms/step - loss: 0.0329
Epoch 7/20
219/219 [==============================] - 2s 7ms/step - loss: 0.0322
Epoch 8/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0316
Epoch 9/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0309
Epoch 10/20
219/219 [==============================] - 2s 8ms/step - loss: 0.0307
Epoch 11/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0300
Epoch 12/20
219/219 [==============================] - 2s 7ms/step - loss: 0.0295
Epoch 13/20
219/219 [====

Die TimeDistributed-Schicht ermöglicht es, eine Schicht (wie z.B. eine Dichteschicht) auf jeden Zeitschritt dieser Sequenz anzuwenden. Das bedeutet, dass die Schicht auf jeden einzelnen Datenpunkt in der Sequenz angewendet wird, anstatt die gesamte Sequenz als Ganzes zu betrachten.

In Ihrem Code wird die TimeDistributed-Schicht verwendet, um eine Dichteschicht (Dense-Schicht) auf jeden Zeitschritt der Ausgabe einer vorherigen SimpleRNN-Schicht anzuwenden. Dies erlaubt dem Modell, auf jede Zeitstufe der Ausgabe der vorherigen Schicht individuell zu reagieren und Muster in den Daten zu erkennen.

Insgesamt ermöglicht die Verwendung der TimeDistributed-Schicht eine feinere Steuerung und Anpassung des Modells an die Daten, insbesondere in Sequenzdatensätzen wie Zeitreihen oder Textdaten.

In [37]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape = [None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(loss="mse", optimizer=optimizer, metrics=[last_time_step_mse])
model.fit(X_train, Y_train, epochs=20)

Epoch 1/20


/Users/riccardo/anaconda3/envs/model/lib/python3.9/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


219/219 [==============================] - 2s 6ms/step - loss: 0.0505 - last_time_step_mse: 0.0397
Epoch 2/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0373 - last_time_step_mse: 0.0246
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0296 - last_time_step_mse: 0.0158
Epoch 4/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0264 - last_time_step_mse: 0.0132
Epoch 5/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0237 - last_time_step_mse: 0.0110
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0222 - last_time_step_mse: 0.0099
Epoch 7/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0201 - last_time_step_mse: 0.0081
Epoch 8/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0194 - last_time_step_mse: 0.0076
Epoch 9/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0191 - last_time_step_mse: 0.0075
Epoch 10/20
219/219 [

# ***Das Gradientenverfahren***

- Allgemeiner Algoritmus zur Optimierung, der optimale Lösungen für eine Vielzahl von Fragestellungen ermitteln kann.
Der Grundgedanke beim Gradientenverfahren ist, die Parameter iterativ so zu verändern das eine Kostenfunktion minimiert wird.

# Gegen instable Gradienten kämpfen 
- Hyperparameter 
    - Droputout
    - recurrent_dropout
abschwächung der instablien Gradienten und das RNN effizienter Trainieren.


### ***Batch Normalization nur bei der Eingabe also Input Layer***
#### Das Problem des Kurzzeitgedächtnisses

Aufgrund der Transformation, die die Daten auf ihrem Weg durch ein RNN durchlaufen, gehen bei jedem Zeitschritt manche Informationen verloren. Nach einiger Zeit enthält der Status RNN keinerlei Spuren der ursprünglichen Eingaben mehr. 
>BSP: Stellen wir uns vor jemand muss ein sehr langen Satz übersetzten - wenn Sie ihn fertig gelesen haben weiß sie nicht mehr, womit er begann.
Um das Problem zu beheben wurde eine Langzeitgedächtnis vorgestellt mit sehr großem erfolg.

### Long Short Term Memory (LSTM)
- Wie eine Blackbox vorstellen. 
- Training wird schneller Konvergieren und wird langfrisitge Abhängigkeiten erkennen in Daten.

<img src="/Users/riccardo/Desktop/Repositorys_Github/Training/Scripts/Models/pictures_expl/Rnn_LSTM_celle.png" width="500" />

In [38]:
model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape = [None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

2024-03-06 16:36:27.025506: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-06 16:36:27.026156: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-06 16:36:27.026797: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [39]:
model.compile(loss = 'mean_squared_error',optimizer='adam')
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 20)          1760      
                                                                 
 lstm_1 (LSTM)               (None, None, 20)          3280      
                                                                 
 time_distributed_2 (TimeDis  (None, None, 10)         210       
 tributed)                                                       
                                                                 
Total params: 5,250
Trainable params: 5,250
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(X_train, Y_train,  epochs=20)

Epoch 1/20


2024-03-06 16:43:38.884455: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-06 16:43:38.885753: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-06 16:43:38.886394: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

219/219 [==============================] - 4s 13ms/step - loss: 0.0798
Epoch 2/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0476
Epoch 3/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0391
Epoch 4/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0346
Epoch 5/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0322
Epoch 6/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0307
Epoch 7/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0294
Epoch 8/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0286
Epoch 9/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0278
Epoch 10/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0270
Epoch 11/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0264
Epoch 12/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0260
Epoch 13/20
219/219 [===

## Gated-Rucurrent- Unit (GRU- ) GRU-Zellen
- Ist eine vereinfachte Variante der LSTM - Zelle, die aber genauso gut zu funkoniert

